# Crossentropy method

강화학습을 crossentropy 메서드를 가지고 푸는 것을 보여줄 것이다.

In [1]:
#서버에서 이것을 돌리면 XVFB가 시작되어 질 것이다.
import os
print(os.environ.get("DISPLAY"))
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    print("DISPLAY")
    !bash ../xvfb start
    %env DISPLAY=:1


None
DISPLAY
bash: ../xvfb: No such file or directory
env: DISPLAY=:1


In [2]:
import gym
import numpy as np, pandas as pd

env = gym.make("Taxi-v2")
env.reset()
env.render()

[2017-05-14 19:58:55,009] Making new env: Taxi-v2


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
n_states = env.observation_space.n
n_actions = env.action_space.n

print("n_states=%i, n_actions=%i"%(n_states,n_actions))

n_states=500, n_actions=6


# 스토캐스틱 정책 만들기

이제 우리의 정책은 확률 분포가 되어야 한다.

policy[s,a] = P(액션 a를 고르다| 상태 s에서)

상태와 액션을 정수 표현으로 사용하기 때문애 정책을 나타내기 위해 2차원 배열을 사용할 수 있다.
정책을 uniform하게 초기화 하자




In [4]:
policy = np.ones((n_states, n_actions)) * 1./n_actions
print(policy)

[[ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 ..., 
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [5]:
assert type(policy) in (np.ndarray,np.matrix)
assert np.allclose(policy,1./n_actions)
assert np.allclose(np.sum(policy,axis=1), 1)

# 게임을 하자~
이전 처럼, 그러나 모든 상태와 액션을 기록한다.

In [6]:
def generate_session(t_max=10**4):
    """
    t_max 틱 동안 또는 끝까지 게임을 하자.
    returns: 상태 리스트, 액션 리스트 그리고 보상 합
    """
    states,actions = [],[]
    total_reward = 0.
    
    s = env.reset()
    
    for t in range(t_max):
        a = np.random.choice(n_actions, p=policy[s])
        new_s,r,done,info = env.step(a)
        states.append(s)
        actions.append(a)
        total_reward += r
        
        s = new_s
        if done:
            break
    return states, actions,total_reward


In [7]:
s,a,r = generate_session()
assert type(s) == type(a) == list
assert len(s) == len(a)
assert type(r) is float


# 학습 루프(Training loop)
섹션을 생성하고, N 베스트를 선택하고 그것에 학습하자


In [8]:
x=np.random.uniform(10,size=(1000))-5.0
# numpy.percentile(a, q, axis=None, out=None, overwrite_input=False, interpolation='linear', keepdims=False)[source]
# Compute the qth percentile of the data along the specified axis.
pcen = np.percentile(x, 70)
print(pcen)

2.27963238394


In [10]:
n_samples = 250 # 샘플링수.
percentile = 50  # 아주 높은 보상을 가지고 섹션의 이 퍼센트를 가져라.
smoothing = 0.1  # 안정적인 분포를 위해 모든 카운드에 이것을 더해라.
alpha = 0.5

total_reward = []
total_threshold = []

for i in range(100):
    
    #%time sessions = <generate n_samples sessions>]
    sessions = [generate_session() for _ in range(n_samples)]
    #print(sessions)
    
    batch_states,batch_actions,batch_rewards = map(np.array,zip(*sessions))
    
    #샘플에서 percentile를 고르자
    threshold = np.percentile(batch_rewards, percentile)
    
    # 보상이 threshold보다 큰 state, actions를 선택하자
    indexes = [i for i, reward in enumerate(batch_rewards) if reward >= threshold]
    elite_states = [batch_states[i] for i in indexes]
    elite_actions = [batch_actions[i] for i in indexes]
    #print(elite_states)
    
    elite_states, elite_actions = map(np.concatenate,[elite_states,elite_actions])
    
    # 엘리트 상태로 부터 액션을 카운티하자
    elite_counts = np.zeros_like(policy)+smoothing
    
    for state, action in zip(elite_states, elite_actions):
        elite_counts[state][action] += 1.0

    #정책을 노멀라이징하자
    elites_count_sum = np.sum(elite_counts, axis=-1)
    #policy = alpha*np.array([elite_counts[i]/s for i,s in enumerate(elites_count_sum)]) + (1.0-alpha)*policy
    policy = [elite_counts[i]/s for i,s in enumerate(elites_count_sum)]
    
    print("mean reward = %.5f\tthreshold = %.1f"%(np.mean(batch_rewards),threshold))
    total_reward.append(np.mean(batch_rewards))
    total_threshold.append(threshold)
    

mean reward = -211.74800	threshold = -12.0
mean reward = -204.75200	threshold = -17.5
mean reward = -160.49600	threshold = -13.5
mean reward = -174.10000	threshold = -14.5
mean reward = -196.41600	threshold = -19.0
mean reward = -178.09200	threshold = -17.0
mean reward = -187.76400	threshold = -13.5
mean reward = -219.45200	threshold = -21.0
mean reward = -173.10800	threshold = -11.5
mean reward = -192.92400	threshold = -17.0
mean reward = -225.34800	threshold = -25.0
mean reward = -221.05200	threshold = -26.5
mean reward = -193.76400	threshold = -22.5
mean reward = -223.32400	threshold = -23.5
mean reward = -195.17200	threshold = -17.5
mean reward = -230.38400	threshold = -30.5
mean reward = -207.18800	threshold = -26.0
mean reward = -200.83200	threshold = -24.0
mean reward = -188.70400	threshold = -18.0
mean reward = -188.32400	threshold = -16.0
mean reward = -189.27600	threshold = -11.5
mean reward = -197.08000	threshold = -22.5
mean reward = -173.16800	threshold = -16.5
mean reward

# Approximate (deep) crossentropy method

이 섹션에서, 연속적인 액션 공간 게임에 대해 뉴럴 네트웍 정책을 학습할 것이다.

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")
env.reset()
n_actions = env.action_space.n

plt.imshow(env.render("rgb_array"))

[2017-05-14 20:02:25,017] Making new env: CartPole-v0


NoSuchDisplayException: Cannot connect to "None"

In [12]:
#에이젼트 생성하자
from sklearn.neural_network import MLPClassifier
agent = MLPClassifier(hidden_layer_sizes=(20,20),
                      activation='tanh',
                      warm_start=True, #keep progress between .fit(...) calls
                      max_iter=1 #make only 1 iteration on each .fit(...)
                     )

# 에이젼트 초기화하자
agent.fit([env.reset()]*n_actions,range(n_actions));

/hanmail/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [13]:
def generate_session(t_max=1000):
    
    states,actions = [],[]
    total_reward = 0
    
    s = env.reset()
    for t in range(t_max):
        
        # 액션 확률 배열을 예측하자
        probs = agent.predict_proba([s])[0] 
        
        # probs확률에 따라서 액션을 선택하자
        a = np.random.choice(n_actions, p=probs)
        
        new_s,r,done,info = env.step(a)
        
        #record sessions like you did before
        states.append(s)
        actions.append(a)
        total_reward+=r
        s = new_s
        if done: break
    return states,actions,total_reward

In [14]:
n_samples = 100
percentile = 70
smoothing = 0.01

for i in range(100):
    #generate new sessions
    sessions = [generate_session() for _ in range(n_samples)]

    batch_states,batch_actions,batch_rewards = map(np.array,zip(*sessions))
    #batch_states: a list of lists of states in each session
    #batch_actions: a list of lists of actions in each session
    #batch_rewards: a list of floats - total rewards at each session

    #샘플에서 percentile를 고르자
    threshold = np.percentile(batch_rewards, percentile) # <select percentile of your samples> 

    # 보상이 threshold보다 큰 state, actions를 선택하자
    indexes = [i for i, reward in enumerate(batch_rewards) if reward >= threshold]
    elite_states = [batch_states[i] for i in indexes]
    elite_actions = [batch_actions[i] for i in indexes]
    
    elite_states, elite_actions = map(np.concatenate,[elite_states,elite_actions])
    #elite_states: a list of states from top games
    #elite_actions: a list of actions from top games
    
    #<fit agent to predict elite_actions(y) from elite_states(X)>
    agent.fit(elite_states, elite_actions);

    print("mean reward = %.5f\tthreshold = %.1f"%(np.mean(batch_rewards),threshold))


mean reward = 14.32000	threshold = 15.3
mean reward = 14.33000	threshold = 15.3
mean reward = 15.64000	threshold = 17.0
mean reward = 18.54000	threshold = 20.6
mean reward = 18.01000	threshold = 22.0
mean reward = 20.32000	threshold = 23.0
mean reward = 20.56000	threshold = 23.0
mean reward = 23.39000	threshold = 26.0
mean reward = 28.66000	threshold = 32.0
mean reward = 24.23000	threshold = 27.3
mean reward = 26.88000	threshold = 31.0
mean reward = 28.74000	threshold = 36.0
mean reward = 32.05000	threshold = 36.0
mean reward = 32.84000	threshold = 39.6
mean reward = 40.06000	threshold = 47.0
mean reward = 43.50000	threshold = 54.5
mean reward = 53.28000	threshold = 62.3
mean reward = 52.64000	threshold = 65.0
mean reward = 63.80000	threshold = 78.9
mean reward = 69.44000	threshold = 91.0
mean reward = 97.17000	threshold = 119.9
mean reward = 100.58000	threshold = 128.0
mean reward = 124.38000	threshold = 169.0
mean reward = 135.58000	threshold = 183.6
mean reward = 161.10000	threshold

# 결과

In [15]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(env,directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()
#unwrap 
env = env.env.env
#upload to gym
#gym.upload("./videos/",api_key="<your_api_key>") #you'll need me later

#Warning! If you keep seeing error that reads something like"DoubleWrapError",
#run env=gym.make("CartPole-v0");env.reset();


[2017-05-14 20:06:38,953] Clearing 3 monitor files from previous run (because force=True was provided)
[2017-05-14 20:06:38,956] Starting new video recorder writing to /data1/users/yrbahn/work/RL_study/Practical_RL/videos/openaigym.video.0.22333.video000000.mp4


NameError: name 'base' is not defined

In [109]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices